In [1]:
import openmeteo_requests
import requests_cache
import torch 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from retry_requests import retry
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.api import VAR

import importlib
import helperfunctions.preprocessing as prep

In [2]:
# Define Constants
VAL_START = "2023-01-01"
TEST_START = "2024-01-01"

SOLAR_PATH = "data/energy_charts.csv"
WEATHER_PATH = "data/weather.csv"

DATE_COL = "date"

# 1. Cleaning Solar Data

In [37]:
# Setup
solar_ts = pd.read_csv(SOLAR_PATH, sep=",", header=0)
solar_ts["Datum"] = pd.to_datetime(solar_ts["Datum"], utc=True)
solar_ts.rename(columns={"Datum": "date"}, inplace=True)

# Split
train_ts, val_ts, test_ts = prep.split_ts(solar_ts, VAL_START, TEST_START, DATE_COL)
train_ts.info()
val_ts.info()
test_ts.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 280516 entries, 2014-12-31 23:00:00+00:00 to 2022-12-31 23:45:00+00:00
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Leistung  280516 non-null  float64
dtypes: float64(1)
memory usage: 4.3 MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35040 entries, 2023-01-01 00:00:00+00:00 to 2023-12-31 23:45:00+00:00
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Leistung  35040 non-null  float64
dtypes: float64(1)
memory usage: 547.5 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12038 entries, 2024-01-01 00:00:00+00:00 to 2024-05-05 09:15:00+00:00
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Leistung  12038 non-null  float64
dtypes: float64(1)
memory usage: 188.1 KB


## 1.1. Leap Years


In [40]:
# Handle Leap Years
for split, ts in zip(["train", "val", "test"], [train_ts, val_ts, test_ts]):    
    ts = prep.handle_leap_years(ts)    

# Pivot table and check unique days
for split, ts in zip(["train", "val", "test"], [train_ts, val_ts, test_ts]):
    pv = pd.pivot_table(
        ts,
        index=["month", "adjusted_dayofyear"],
        columns=ts.index.year,
        values="Leistung",
        aggfunc="sum"
    )
    
    unique_days_count = ts['adjusted_dayofyear'].nunique()
    print(f"Number of unique days in the {split} dataset: {unique_days_count}")

Number of unique days in the train dataset: 365
Number of unique days in the val dataset: 365
Number of unique days in the test dataset: 125


## 1.2. Missing Values, Missing Timestamps, Duplicated Entries

In [39]:
# Check for missing values
for split, ts in zip(["train", "val", "test"], [train_ts, val_ts, test_ts]):
    print(f"Missing values in {split} dataset: {ts.isnull().sum().sum()}")
    
# Check for duplicated dates
for ts in [train_ts, val_ts, test_ts]:
    if ts.index.duplicated().any():
        print("Duplicates found:")
        print(ts[ts.index.duplicated(keep=False)])

# Keep the first occurrence of each duplicate
for i, ts in enumerate([train_ts, val_ts, test_ts]):
    ts = ts[~ts.index.duplicated(keep='first')]
    if i == 0:
        train_ts = ts
    elif i == 1:
        val_ts = ts
    else:
        test_ts = ts

# Confirm that there are no duplicates anymore
for split, ts in zip(["train", "val", "test"], [train_ts, val_ts, test_ts]):
    if not ts.index.duplicated().any():
        print(f"No duplicates found in {split} dataset")
    else:
        print(ts[ts.index.duplicated(keep=False)])
        
# Check for missing timestamps
for split, ts in zip(["train", "val", "test"], [train_ts, val_ts, test_ts]):
    prep.check_continuity(ts, split)

# Interpolate missing timestamps
importlib.reload(prep)

time_series_list = [train_ts, val_ts, test_ts]

for i in range(len(time_series_list)):
    time_series_list[i] = prep.interpolate_missing_timestamps(time_series_list[i])

train_ts, val_ts, test_ts = time_series_list
train_ts.head()

Missing values in train dataset: 0
Missing values in val dataset: 0
Missing values in test dataset: 0
Duplicates found:
                           Leistung  adjusted_dayofyear  month
date                                                          
2015-03-01 00:00:00+00:00       0.0                  60      3
2015-03-01 00:15:00+00:00       0.0                  60      3
2015-03-01 00:30:00+00:00       0.0                  60      3
2015-03-01 00:45:00+00:00       0.0                  60      3
2015-03-01 00:00:00+00:00       0.0                  60      3
...                             ...                 ...    ...
2022-09-30 00:45:00+00:00       0.0                 273      9
2022-09-30 00:00:00+00:00       0.0                 273      9
2022-09-30 00:15:00+00:00       0.0                 273      9
2022-09-30 00:30:00+00:00       0.0                 273      9
2022-09-30 00:45:00+00:00       0.0                 273      9

[128 rows x 3 columns]
Duplicates found:
                   

,Leistung,adjusted_dayofyear,month
2014-12-31 23:00:00+00:00,0.0,365.0,12.0
2014-12-31 23:15:00+00:00,0.0,365.0,12.0
2014-12-31 23:30:00+00:00,0.0,365.0,12.0
2014-12-31 23:45:00+00:00,0.0,365.0,12.0
2015-01-01 00:00:00+00:00,0.0,1.0,1.0


## 1.3. Resampling to Hourly Frequency

In [47]:
TODO: fix resampling of mothns

# Resample from 15min to hourly
time_series_list = [train_ts, val_ts, test_ts]
for i in range(len(time_series_list)):
    time_series_list[i] = prep.resample_ts(time_series_list[i])

train_ts, val_ts, test_ts = time_series_list   

# Ensure other columns are not summed up
for ts in [train_ts, val_ts, test_ts]:    
    for column in ts.columns:
        if column != "Leistung":
            ts[column] = ts[column] / 4

train_ts.head(25)

,Leistung,adjusted_dayofyear,month
2014-12-31 23:00:00+00:00,0.0,22.8125,0.7500
2015-01-01 00:00:00+00:00,0.0,0.0625,0.0625
2015-01-01 01:00:00+00:00,0.0,0.0625,0.0625
2015-01-01 02:00:00+00:00,0.0,0.0625,0.0625
2015-01-01 03:00:00+00:00,0.0,0.0625,0.0625
2015-01-01 04:00:00+00:00,0.0,0.0625,0.0625
2015-01-01 05:00:00+00:00,0.0,0.0625,0.0625
2015-01-01 06:00:00+00:00,0.0,0.0625,0.0625
2015-01-01 07:00:00+00:00,227.4,0.0625,0.0625
2015-01-01 08:00:00+00:00,3309.8,0.0625,0.0625
